# 04_nuplan_dataset_intro_and_animation

このノートは、nuPlan Datasetの**データ構造**と**取り扱い方法**を初心者向けに整理し、
nuPlan devkitのツールを使って**シナリオ単位のアニメーション**を作るところまでを扱います。

- Q1: nuPlan Datasetのデータ構造と取り扱い方法の解説
- Q2: nuPlanのツールを使ったシナリオごとのアニメーション


## nuPlan devkit側の主要クラス（コード参照）

このノートでは、nuPlan devkitの以下のクラス/関数を利用します。

- `NuPlanDB`（`nuplan/database/nuplan_db_orm/nuplandb.py`）: .dbファイルをロードし、テーブルにアクセスするORM
- `NuPlanScenarioBuilder`（`nuplan/planning/scenario_builder/nuplan_db/nuplan_scenario_builder.py`）: ログDBからシナリオを構築
- `NuPlanScenario`（`nuplan/planning/scenario_builder/nuplan_db/nuplan_scenario.py`）: シナリオ内のLidarトークン列やEgo状態を提供
- `LidarPc.render`（`nuplan/database/nuplan_db_orm/lidar_pc.py`）: マップ上にLiDAR + アノテーションを描画


## 1. 環境変数と基本パス

nuPlan devkitのコードでは以下の環境変数を使う前提になっています。

- `NUPLAN_DATA_ROOT`: データルート（.dbやsplitsの起点）
- `NUPLAN_MAPS_ROOT`: 地図データのルート
- `NUPLAN_MAP_VERSION`: 地図バージョン


In [2]:
import os
from pathlib import Path

NUPLAN_DATA_ROOT = os.getenv("NUPLAN_DATA_ROOT")
NUPLAN_MAPS_ROOT = os.getenv("NUPLAN_MAPS_ROOT")
NUPLAN_MAP_VERSION = os.getenv("NUPLAN_MAP_VERSION")

print("NUPLAN_DATA_ROOT:", NUPLAN_DATA_ROOT)
print("NUPLAN_MAPS_ROOT:", NUPLAN_MAPS_ROOT)
print("NUPLAN_MAP_VERSION:", NUPLAN_MAP_VERSION)

data_root = Path(NUPLAN_DATA_ROOT) if NUPLAN_DATA_ROOT else None
maps_root = Path(NUPLAN_MAPS_ROOT) if NUPLAN_MAPS_ROOT else None

if data_root is None or not data_root.exists():
    print("❌ NUPLAN_DATA_ROOT が未設定、または存在しません")
if maps_root is None or not maps_root.exists():
    print("❌ NUPLAN_MAPS_ROOT が未設定、または存在しません")


NUPLAN_DATA_ROOT: /nuplan/dataset
NUPLAN_MAPS_ROOT: /nuplan/dataset/maps
NUPLAN_MAP_VERSION: None


## 2. データ構造（最小限）

nuPlanのログは **`.db` ファイル（SQLite）** で構成され、`NuPlanDB`が読み込みます。
`NuPlanScenarioBuilder`は、データルート・マップ・センサーデータを指定してシナリオを組み立てます。

以下のセルでは、`NUPLAN_DATA_ROOT`配下から `splits` ディレクトリ候補を探し、
利用可能な `.db` を確認します。


In [3]:
from typing import List

def find_splits_dirs(root: Path) -> List[Path]:
    # splits ディレクトリを探す（見つからない場合は空）
    if not root.exists():
        return []
    return sorted([p for p in root.rglob("splits") if p.is_dir()])

splits_dirs = find_splits_dirs(data_root) if data_root else []
print("splits candidates:")
for p in splits_dirs:
    print(" -", p)

# 先頭のsplits配下から、.dbがあるサブディレクトリを選ぶ
split_dir = None
for splits_root in splits_dirs:
    subdirs = sorted([d for d in splits_root.iterdir() if d.is_dir()])
    for d in subdirs:
        if list(d.glob('*.db')):
            split_dir = d
            break
    if split_dir:
        break

print("selected split_dir:", split_dir)
db_files = sorted([str(p) for p in split_dir.glob('*.db')]) if split_dir else []
print("db files:", len(db_files))


splits candidates:
 - /nuplan/dataset/nuplan-v1.1/splits
selected split_dir: /nuplan/dataset/nuplan-v1.1/splits/mini
db files: 64


## 3. NuPlanScenarioBuilderでシナリオを作る

`NuPlanScenarioBuilder`は `.db` ファイルからシナリオを構築します。
`ScenarioFilter`で対象を絞り込みます（ここでは最小例として1件に制限）。


In [4]:
from nuplan.planning.scenario_builder.nuplan_db.nuplan_scenario_builder import NuPlanScenarioBuilder
from nuplan.planning.scenario_builder.scenario_filter import ScenarioFilter
from nuplan.planning.utils.multithreading.worker_sequential import Sequential

if not db_files:
    print("❌ .dbファイルが見つかりません")
else:
    scenario_builder = NuPlanScenarioBuilder(
        data_root=str(data_root),
        map_root=str(maps_root),
        sensor_root=str(data_root / 'sensor_blobs'),
        db_files=db_files,
        map_version=NUPLAN_MAP_VERSION,
        include_cameras=False,
    )

    scenario_filter = ScenarioFilter(
        scenario_types=None,
        scenario_tokens=None,
        log_names=None,
        map_names=None,
        num_scenarios_per_type=None,
        limit_total_scenarios=1,
        timestamp_threshold_s=None,
        ego_displacement_minimum_m=None,
        expand_scenarios=False,
        remove_invalid_goals=False,
        shuffle=False,
    )

    worker = Sequential()
    scenarios = scenario_builder.get_scenarios(scenario_filter, worker)
    print("scenarios:", len(scenarios))
    if scenarios:
        print("first scenario type:", scenarios[0].scenario_type)


scenarios: 1
first scenario type: stationary


## 4. NuPlanScenarioから取り出せる情報

`NuPlanScenario`は以下のような情報を提供します。

- `get_number_of_iterations()` : シナリオ内のLidarフレーム数
- `get_scenario_tokens()` : LidarPCトークン列
- `get_ego_state_at_iteration(i)` : i番目のEgoState


In [5]:
if 'scenarios' in globals() and scenarios:
    scenario = scenarios[0]
    print("iterations:", scenario.get_number_of_iterations())
    tokens = scenario.get_scenario_tokens()
    print("tokens:", len(tokens))
    print("first token:", tokens[0])
    ego_state = scenario.get_ego_state_at_iteration(0)
    print("ego_state@0:", ego_state)


iterations: 401
tokens: 401
first token: 00002b2436735b10
ego_state@0: <nuplan.common.actor_state.ego_state.EgoState object at 0x7bbcd16a6130>


## 5. シナリオごとのアニメーション（nuPlanの描画ツール）

`LidarPc.render` は内部で `render_on_map` を呼び、マップ + 点群 + アノテーションを描画します。
ここでは **1シナリオのLidarトークン列** を使ってアニメーションを作ります。


In [6]:
import matplotlib.pyplot as plt
from matplotlib import animation
from IPython.display import HTML

from nuplan.database.nuplan_db_orm.nuplandb import NuPlanDB
from nuplan.database.nuplan_db_orm.lidar_pc import LidarPc
from nuplan.common.maps.nuplan_map.map_factory import get_maps_db

if 'scenario' not in globals():
    print("❌ scenario が未生成です")
else:
    # NuPlanScenario は内部でダウンロード済みのログパスを保持している
    log_path = scenario._log_file  # 内部属性: ローカルに解決済みのパス

    maps_db = get_maps_db(str(maps_root), NUPLAN_MAP_VERSION)
    db = NuPlanDB(data_root=str(data_root), load_path=log_path, maps_db=maps_db)

    tokens = scenario.get_scenario_tokens()
    frame_tokens = tokens[:50]  # 描画コストを抑えるため先頭50フレームのみ

    fig, ax = plt.subplots(1, 1, figsize=(6, 6))

    def render_frame(i):
        token = frame_tokens[i]
        lidar_pc = db.session.query(LidarPc).filter(LidarPc.token == token).one()
        ax.clear()
        lidar_pc.render(
            db,
            render_map_raster=True,
            render_vector_map=False,
            render_future_ego_poses=False,
            axes_limit=80.0,
            ax=ax,
        )
        ax.set_title(f"{i+1}/{len(frame_tokens)} token={token}")
        return ax

    anim = animation.FuncAnimation(
        fig, render_frame, frames=len(frame_tokens), interval=100, blit=False
    )
    HTML(anim.to_jshtml())


BlobStoreKeyNotFound: [Errno 2] No such file or directory: '/nuplan/dataset/maps/None.json'

## 補足

- `LidarPc.render` は `render_on_map` を利用し、地図やアノテーションの表示を行います。
- `axes_limit` を小さくすると表示範囲を絞れます。
- フレーム数が多い場合は `frame_tokens` を短くして負荷を調整してください。
